In [ ]:
# Install required packages if needed
# For GPU acceleration on Kaggle, CuML is pre-installed
# On local machines: pip install cuml-cu11 cupy-cuda11x
# !pip install pandas numpy scikit-learn tqdm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
import pickle
from collections import Counter, defaultdict
import re
import os
import warnings
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')

# GPU Detection and Library Import
try:
    import cupy as cp
    from cuml.linear_model import LogisticRegression as cuLogisticRegression
    from cuml.multiclass import OneVsRestClassifier as cuOneVsRestClassifier
    GPU_AVAILABLE = True
    print("✓ GPU libraries loaded successfully!")
    print(f"GPU Device: {cp.cuda.Device()}")
    print(f"GPU Memory: {cp.cuda.Device().mem_info[1] / 1e9:.2f} GB total")
except ImportError:
    GPU_AVAILABLE = False
    from sklearn.linear_model import LogisticRegression
    from sklearn.multiclass import OneVsRestClassifier
    print("⚠ GPU libraries not available, using CPU (sklearn)")

print("Libraries imported successfully!")

## 1. Configuration

In [ ]:
# Paths for Kaggle environment
DATA_DIR = "/kaggle/input/cafa-5-protein-function-prediction"
OUTPUT_DIR = "/kaggle/working"

# Model parameters
TOP_N_TERMS = 600  # Number of most common GO terms to predict
TEST_SIZE = 0.15  # Validation split ratio
RANDOM_STATE = 42
PREDICTION_THRESHOLD = 0.3  # Threshold for final predictions

# Logistic Regression parameters
LR_MAX_ITER = 1000
LR_C = 0.5  # Regularization strength
LR_SOLVER = 'lbfgs'

print(f"Configuration set!")
print(f"Top GO terms to predict: {TOP_N_TERMS}")
print(f"Prediction threshold: {PREDICTION_THRESHOLD}")

## 2. Load Training Data

In [ ]:
# Load training terms
print("Loading training data...")
train_terms = pd.read_csv(os.path.join(DATA_DIR, 'Train', 'train_terms.tsv'), sep='\t')
print(f"Loaded {len(train_terms)} term annotations")
print(train_terms.head())

In [ ]:
# Load taxonomy (optional - can be used for additional features)
train_taxonomy = pd.read_csv(
    os.path.join(DATA_DIR, 'Train', 'train_taxonomy.tsv'), 
    sep='\t', 
    header=None, 
    names=['EntryID', 'TaxonID']
)
print(f"Loaded {len(train_taxonomy)} taxonomy entries")

## 3. Parse FASTA Sequences

In [ ]:
def parse_fasta(fasta_path):
    """
    Parse FASTA file and return dict of protein_id -> sequence
    """
    sequences = {}
    current_id = None
    current_seq = []
    
    with open(fasta_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # Save previous sequence
                if current_id is not None:
                    sequences[current_id] = ''.join(current_seq)
                
                # Extract ID from header (format: >sp|ID|... or >ID taxon)
                match = re.search(r'\|([A-Z0-9]+)\|', line)
                if match:
                    current_id = match.group(1)
                else:
                    current_id = line[1:].split()[0]
                current_seq = []
            else:
                current_seq.append(line)
        
        # Don't forget the last sequence
        if current_id is not None:
            sequences[current_id] = ''.join(current_seq)
    
    return sequences

In [ ]:
# Parse training sequences
print("Parsing training FASTA sequences...")
train_sequences = parse_fasta(os.path.join(DATA_DIR, 'Train', 'train_sequences.fasta'))
print(f"Parsed {len(train_sequences)} protein sequences")

# Show example
sample_id = list(train_sequences.keys())[0]
print(f"\nExample: {sample_id}")
print(f"Sequence length: {len(train_sequences[sample_id])}")
print(f"First 50 amino acids: {train_sequences[sample_id][:50]}")

## 4. Feature Extraction

In [ ]:
def extract_features(sequence):
    """
    Extract comprehensive features from amino acid sequence:
    - Amino acid composition (20 features)
    - Sequence length and log-length
    - Physicochemical properties
    - Dipeptide frequencies (20 most common)
    - N-terminal and C-terminal composition
    """
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    features = {}
    
    # Basic properties
    seq_len = len(sequence)
    features['length'] = seq_len
    features['log_length'] = np.log1p(seq_len)
    
    if seq_len == 0:
        # Return zero features for empty sequences
        for aa in amino_acids:
            features[f'aa_{aa}'] = 0.0
        
        # Physicochemical properties
        for prop in ['hydrophobic', 'polar', 'charged', 'positive', 'negative', 
                     'aromatic', 'aliphatic', 'tiny', 'small', 'large']:
            features[f'{prop}_ratio'] = 0.0
        
        # Dipeptides
        common_dipeptides = ['AA', 'AL', 'LL', 'LA', 'GG', 'VA', 'AV', 'EE', 'KK', 'RR',
                            'SS', 'TT', 'PP', 'DD', 'NN', 'QQ', 'FF', 'WW', 'YY', 'CC']
        for dp in common_dipeptides:
            features[f'di_{dp}'] = 0.0
        
        # Terminal composition
        for aa in ['A', 'M', 'G', 'S', 'T', 'L', 'K', 'R']:
            features[f'n_term_{aa}'] = 0.0
            features[f'c_term_{aa}'] = 0.0
        
        return features
    
    # Amino acid composition (normalized)
    aa_counts = Counter(sequence)
    for aa in amino_acids:
        features[f'aa_{aa}'] = aa_counts.get(aa, 0) / seq_len
    
    # Physicochemical properties
    hydrophobic = 'AVILMFWP'
    polar = 'STNQCY'
    positive = 'RK'
    negative = 'DE'
    aromatic = 'FWY'
    aliphatic = 'AILV'
    tiny = 'AGSCT'
    small = 'ABCDGNPSTV'
    large = 'EFHIKLMQRWY'
    
    features['hydrophobic_ratio'] = sum(aa_counts.get(aa, 0) for aa in hydrophobic) / seq_len
    features['polar_ratio'] = sum(aa_counts.get(aa, 0) for aa in polar) / seq_len
    features['charged_ratio'] = sum(aa_counts.get(aa, 0) for aa in positive + negative) / seq_len
    features['positive_ratio'] = sum(aa_counts.get(aa, 0) for aa in positive) / seq_len
    features['negative_ratio'] = sum(aa_counts.get(aa, 0) for aa in negative) / seq_len
    features['aromatic_ratio'] = sum(aa_counts.get(aa, 0) for aa in aromatic) / seq_len
    features['aliphatic_ratio'] = sum(aa_counts.get(aa, 0) for aa in aliphatic) / seq_len
    features['tiny_ratio'] = sum(aa_counts.get(aa, 0) for aa in tiny) / seq_len
    features['small_ratio'] = sum(aa_counts.get(aa, 0) for aa in small) / seq_len
    features['large_ratio'] = sum(aa_counts.get(aa, 0) for aa in large) / seq_len
    
    # Dipeptide frequencies
    dipeptides = [sequence[i:i+2] for i in range(len(sequence)-1)]
    dipeptide_counts = Counter(dipeptides)
    
    common_dipeptides = ['AA', 'AL', 'LL', 'LA', 'GG', 'VA', 'AV', 'EE', 'KK', 'RR',
                        'SS', 'TT', 'PP', 'DD', 'NN', 'QQ', 'FF', 'WW', 'YY', 'CC']
    for dp in common_dipeptides:
        features[f'di_{dp}'] = dipeptide_counts.get(dp, 0) / max(len(dipeptides), 1)
    
    # N-terminal and C-terminal composition
    terminal_len = max(10, int(seq_len * 0.1))
    n_terminal = sequence[:terminal_len]
    c_terminal = sequence[-terminal_len:]
    
    n_term_counts = Counter(n_terminal)
    c_term_counts = Counter(c_terminal)
    
    for aa in ['A', 'M', 'G', 'S', 'T', 'L', 'K', 'R']:
        features[f'n_term_{aa}'] = n_term_counts.get(aa, 0) / len(n_terminal)
        features[f'c_term_{aa}'] = c_term_counts.get(aa, 0) / len(c_terminal)
    
    return features

print("Feature extraction function defined!")

In [ ]:
# Extract features from all sequences
print("Extracting features from sequences...")
feature_data = []
entry_ids = []

for entry_id, sequence in tqdm(train_sequences.items(), desc="Extracting features"):
    features = extract_features(sequence)
    feature_data.append(features)
    entry_ids.append(entry_id)

# Convert to DataFrame
X_df = pd.DataFrame(feature_data, index=entry_ids)
print(f"\nExtracted features shape: {X_df.shape}")
print(f"Feature columns: {X_df.columns.tolist()[:10]}...")

In [ ]:
# Standardize features
print("Standardizing features...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df.values)
print(f"Feature scaling complete. Shape: {X_scaled.shape}")

## 5. Prepare Labels (GO Terms)

In [ ]:
# Prepare labels - multi-label format
print("Preparing GO term labels...")
entry_to_terms = defaultdict(set)

for idx, row in tqdm(train_terms.iterrows(), total=len(train_terms), desc="Building label mapping"):
    entry_to_terms[row['EntryID']].add(row['term'])

# Convert sets to lists
entry_to_terms = {k: list(v) for k, v in entry_to_terms.items()}

print(f"Built mapping for {len(entry_to_terms)} proteins")

In [ ]:
# Filter to only include entries with both sequences and GO terms
valid_entry_ids = [eid for eid in entry_ids if eid in entry_to_terms]
print(f"Found {len(valid_entry_ids)} entries with both sequences and GO terms")

# Prepare final datasets
X = X_scaled[X_df.index.isin(valid_entry_ids)]
y_labels = [entry_to_terms[eid] for eid in valid_entry_ids]

print(f"\nFinal training data shape: {X.shape}")
print(f"Number of samples with labels: {len(y_labels)}")

In [ ]:
# Analyze label distribution
term_counts = Counter([term for terms in y_labels for term in terms])
print(f"\nTotal unique GO terms: {len(term_counts)}")
print(f"\nTop 20 most common GO terms:")
for term, count in term_counts.most_common(20):
    print(f"  {term}: {count}")

In [ ]:
# Filter to top N most common terms
top_terms = [term for term, count in term_counts.most_common(TOP_N_TERMS)]
print(f"\nFocusing on top {TOP_N_TERMS} most common GO terms")

# Filter labels to top terms
filtered_y_labels = [[term for term in terms if term in top_terms] for terms in y_labels]

# Binarize labels
mlb_filtered = MultiLabelBinarizer(classes=top_terms)
y_filtered = mlb_filtered.fit_transform(filtered_y_labels)

print(f"Filtered label matrix shape: {y_filtered.shape}")
print(f"Average labels per protein: {y_filtered.sum() / len(y_filtered):.2f}")
print(f"Sparsity: {1 - (y_filtered.sum() / y_filtered.size):.4f}")

## 6. Train/Test Split

In [ ]:
# Split data for validation
X_train, X_test, y_train, y_test = train_test_split(
    X, y_filtered, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"\nTraining labels per sample: {y_train.sum(axis=1).mean():.2f}")
print(f"Test labels per sample: {y_test.sum(axis=1).mean():.2f}")

## 7. Train Multi-Label Logistic Regression

In [ ]:
# Build model
print("Building multi-label logistic regression model...")
print(f"Training {len(top_terms)} binary classifiers in parallel...")

if GPU_AVAILABLE:
    print("Using GPU acceleration with CuML!")
    model = cuOneVsRestClassifier(
        cuLogisticRegression(
            max_iter=LR_MAX_ITER,
            C=LR_C,
            penalty='l2',
            verbose=0
        ),
        n_jobs=-1
    )
else:
    print("Using CPU with scikit-learn")
    model = OneVsRestClassifier(
        LogisticRegression(
            max_iter=LR_MAX_ITER,
            solver=LR_SOLVER,
            C=LR_C,
            penalty='l2',
            random_state=RANDOM_STATE,
            class_weight='balanced',
            n_jobs=1,
            verbose=0
        ),
        n_jobs=-1
    )

print("Model configured!")

In [ ]:
# Train model
print("\nTraining model... This may take several minutes...")

if GPU_AVAILABLE:
    # Convert to CuPy arrays for GPU training
    X_train_gpu = cp.asarray(X_train)
    y_train_gpu = cp.asarray(y_train)
    model.fit(X_train_gpu, y_train_gpu)
    # Free GPU memory
    del X_train_gpu, y_train_gpu
    cp.get_default_memory_pool().free_all_blocks()
else:
    model.fit(X_train, y_train)

print("\n✓ Training complete!")

## 8. Model Evaluation

In [ ]:
# Make predictions
print("Evaluating model...")

if GPU_AVAILABLE:
    X_test_gpu = cp.asarray(X_test)
    y_pred = cp.asnumpy(model.predict(X_test_gpu))
    y_pred_proba = cp.asnumpy(model.predict_proba(X_test_gpu))
    del X_test_gpu
    cp.get_default_memory_pool().free_all_blocks()
else:
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

print("Predictions complete!")

In [ ]:
# Calculate metrics
f1_samples = f1_score(y_test, y_pred, average='samples', zero_division=0)
precision_samples = precision_score(y_test, y_pred, average='samples', zero_division=0)
recall_samples = recall_score(y_test, y_pred, average='samples', zero_division=0)

f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=0)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)
f1_weighted = f1_score(y_test, y_pred, average='weighted', zero_division=0)

precision_micro = precision_score(y_test, y_pred, average='micro', zero_division=0)
recall_micro = recall_score(y_test, y_pred, average='micro', zero_division=0)

hamming = hamming_loss(y_test, y_pred)

print("\n" + "="*50)
print("MODEL PERFORMANCE")
print("="*50)
print(f"\nSample-based metrics:")
print(f"  F1-Score:  {f1_samples:.4f}")
print(f"  Precision: {precision_samples:.4f}")
print(f"  Recall:    {recall_samples:.4f}")

print(f"\nMicro-averaged metrics:")
print(f"  F1-Score:  {f1_micro:.4f}")
print(f"  Precision: {precision_micro:.4f}")
print(f"  Recall:    {recall_micro:.4f}")

print(f"\nOther metrics:")
print(f"  F1-Macro:    {f1_macro:.4f}")
print(f"  F1-Weighted: {f1_weighted:.4f}")
print(f"  Hamming Loss: {hamming:.4f}")
print("="*50)

In [ ]:
# Threshold analysis
print("\n" + "="*50)
print("THRESHOLD ANALYSIS")
print("="*50)

threshold_results = []
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5]:
    y_pred_thresh = (y_pred_proba >= threshold).astype(int)
    f1_thresh = f1_score(y_test, y_pred_thresh, average='samples', zero_division=0)
    coverage = (y_pred_thresh.sum(axis=1) > 0).mean()
    avg_preds = y_pred_thresh.sum(axis=1).mean()
    
    threshold_results.append({
        'threshold': threshold,
        'f1': f1_thresh,
        'coverage': coverage,
        'avg_predictions': avg_preds
    })
    
    print(f"Threshold {threshold:.1f}: F1={f1_thresh:.4f}, Coverage={coverage:.2%}, Avg predictions={avg_preds:.2f}")

# Find best threshold
best_result = max(threshold_results, key=lambda x: x['f1'])
print(f"\n✓ Best threshold: {best_result['threshold']:.1f} (F1={best_result['f1']:.4f})")
print("="*50)

## 9. Feature Importance Analysis

In [ ]:
# Analyze feature importance for top GO terms
print("\nAnalyzing feature importance for top GO terms...\n")

for i in range(min(5, len(top_terms))):
    term = top_terms[i]
    estimator = model.estimators_[i]
    coef = estimator.coef_[0]
    
    # Get top positive and negative features
    top_pos_idx = np.argsort(coef)[-5:][::-1]
    top_neg_idx = np.argsort(coef)[:5]
    
    print(f"GO Term: {term}")
    print(f"  Top positive features:")
    for idx in top_pos_idx:
        print(f"    {X_df.columns[idx]}: {coef[idx]:.4f}")
    print(f"  Top negative features:")
    for idx in top_neg_idx:
        print(f"    {X_df.columns[idx]}: {coef[idx]:.4f}")
    print()

## 10. Save Model

In [ ]:
# Save model and preprocessing objects
print("Saving model...")

model_data = {
    'model': model,
    'mlb': mlb_filtered,
    'scaler': scaler,
    'feature_names': X_df.columns.tolist(),
    'top_terms': top_terms,
    'threshold': PREDICTION_THRESHOLD,
    'performance': {
        'f1_samples': f1_samples,
        'precision_samples': precision_samples,
        'recall_samples': recall_samples,
        'f1_micro': f1_micro
    }
}

model_path = os.path.join(OUTPUT_DIR, 'logistic_regression_model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model_data, f)

print(f"✓ Model saved to: {model_path}")

## 11. Generate Predictions for Test Set

In [ ]:
# Parse test sequences
print("Loading test sequences...")
test_sequences = parse_fasta(os.path.join(DATA_DIR, 'Test', 'testsuperset.fasta'))
print(f"Loaded {len(test_sequences)} test sequences")

In [ ]:
# Extract features for test sequences
print("Extracting features from test sequences...")
test_feature_data = []
test_entry_ids = []

for entry_id, sequence in tqdm(test_sequences.items(), desc="Extracting test features"):
    features = extract_features(sequence)
    test_feature_data.append(features)
    test_entry_ids.append(entry_id)

# Convert to DataFrame
X_test_df = pd.DataFrame(test_feature_data, index=test_entry_ids)
print(f"Test features shape: {X_test_df.shape}")

# Standardize using training scaler
X_test_scaled = scaler.transform(X_test_df.values)
print(f"Test features scaled")

In [ ]:
# Generate predictions
print("Generating predictions for test set...")

if GPU_AVAILABLE:
    # Process in batches to manage GPU memory
    batch_size = 10000
    n_samples = len(X_test_scaled)
    n_batches = (n_samples + batch_size - 1) // batch_size
    test_pred_proba = np.zeros((n_samples, len(top_terms)))
    
    for i in tqdm(range(n_batches), desc="GPU prediction batches"):
        start = i * batch_size
        end = min(start + batch_size, n_samples)
        X_batch_gpu = cp.asarray(X_test_scaled[start:end])
        test_pred_proba[start:end] = cp.asnumpy(model.predict_proba(X_batch_gpu))
        del X_batch_gpu
        cp.get_default_memory_pool().free_all_blocks()
else:
    test_pred_proba = model.predict_proba(X_test_scaled)

print(f"Predictions generated for {len(test_entry_ids)} proteins")

In [ ]:
# Create submission file
print("Creating submission file...")

submission_rows = []

for i, protein_id in enumerate(tqdm(test_entry_ids, desc="Building submission")):
    # Get predictions above threshold
    probs = test_pred_proba[i]
    pred_indices = np.where(probs >= PREDICTION_THRESHOLD)[0]
    
    if len(pred_indices) == 0:
        # If no predictions above threshold, take top 5
        pred_indices = np.argsort(probs)[-5:][::-1]
    
    for idx in pred_indices:
        term = top_terms[idx]
        score = probs[idx]
        submission_rows.append({
            'EntryID': protein_id,
            'term': term,
            'score': round(float(score), 4)
        })

# Create DataFrame
submission_df = pd.DataFrame(submission_rows)
print(f"\nSubmission contains {len(submission_df)} predictions")
print(f"Unique proteins: {submission_df['EntryID'].nunique()}")
print(f"Unique GO terms: {submission_df['term'].nunique()}")

In [ ]:
# Save submission file
submission_path = os.path.join(OUTPUT_DIR, 'submission.tsv')
submission_df.to_csv(submission_path, sep='\t', index=False, header=False)

print(f"\n✓ Submission saved to: {submission_path}")
print("\nSubmission preview:")
print(submission_df.head(20))

In [ ]:
# Submission statistics
terms_per_protein = submission_df.groupby('EntryID').size()

print("\n" + "="*50)
print("SUBMISSION STATISTICS")
print("="*50)
print(f"Total predictions: {len(submission_df)}")
print(f"Unique proteins: {submission_df['EntryID'].nunique()}")
print(f"Unique GO terms: {submission_df['term'].nunique()}")
print(f"\nScore statistics:")
print(f"  Min:    {submission_df['score'].min():.4f}")
print(f"  Max:    {submission_df['score'].max():.4f}")
print(f"  Mean:   {submission_df['score'].mean():.4f}")
print(f"  Median: {submission_df['score'].median():.4f}")
print(f"\nTerms per protein:")
print(f"  Min:    {terms_per_protein.min()}")
print(f"  Max:    {terms_per_protein.max()}")
print(f"  Mean:   {terms_per_protein.mean():.1f}")
print(f"  Median: {terms_per_protein.median():.1f}")
print("="*50)

## Summary

### Model Performance:
- **Algorithm**: Multi-label Logistic Regression (One-vs-Rest)
- **Features**: Amino acid composition + physicochemical properties
- **GO Terms**: Top 600 most common terms

### Note:
This is a baseline model using simple sequence features. For significantly better performance:
1. Use the **KNN model with ESM-2 embeddings** (see `knn_protein_prediction.ipynb`)
2. ESM-2 embeddings capture much richer protein representations
3. Expected F1 improvement: ~2-3x better with ESM-2

### Files Generated:
- `logistic_regression_model.pkl` - Trained model
- `submission.tsv` - Competition submission file

In [ ]:
print("\n✓ All done! Ready for submission.")